In [ ]:
from common import *
from snotel_sites import SNOTEL

In [ ]:
client = start_cluster(4, 8)
client

In [ ]:
home = PurePath(f'{Path.home()}/shared-cyrosphere/')
year = 'wy2019'

modis = home / 'Rittger_albedo' / year
files_modis = 'WesternUS_20190[3-7]*.nc'

snobal = home / 'iSnobal/output' / year / 'initial_run/data'
files_snobal = 'data20190[3-7]*/smrfOutputs/albedo_*.nc'

In [ ]:
albedo_modis = xr.open_mfdataset(
    modis.joinpath(files_modis).as_posix(),
    mask_and_scale=True,
    parallel=True,
)

# Select MST 11 AM; time of MODIS overpass
albedo_snobal = xr.open_mfdataset(
    snobal.joinpath(files_snobal).as_posix(),
    parallel=True,
).sel(time=datetime.time(18))

albedo_irwin = pd.read_csv(
    home.joinpath('Snotel', '2019-Irwin-BBAlbedo.csv').as_posix(),
    parse_dates=True,
    index_col=0,
    date_parser=lambda x: pd.to_datetime(x, format='%Y-%j'),
)

In [ ]:
irwin_query = dict(
    x=[SNOTEL['Irwin']['lon']], 
    y=[SNOTEL['Irwin']['lat']],
    method='nearest',
)

albedo_modis = albedo_modis.sel(**irwin_query)

albedo_snobal = albedo_snobal.sel(**irwin_query)
albedo_snobal = (albedo_snobal.albedo_vis + albedo_snobal.albedo_ir)/2

albedo_irwin = albedo_irwin[albedo_irwin.index < '2019-08-01']

In [ ]:
figure_opts = dict(figsize=(6,4), dpi=300, tight_layout=True)
water_year = pd.date_range(start='2019-03-01', periods=5, freq='MS')

xTicks = mdates.DateFormatter('%d-%b')

figure, ax = plt.subplots(1, 1, **figure_opts)
figure.set_facecolor('lightgrey')

albedo_modis.albedo_muz.plot(
    ax=ax, 
    c='indigo', 
)
albedo_snobal.plot(
    ax=ax, 
    c='darkorange',
)
albedo_irwin.plot(
    ax=ax, 
    c='steelblue', 
)

ax.legend(['MODIS', 'Modeled', 'In Situ'], loc='lower right');
ax.xaxis.set_major_formatter(xTicks)
ax.set_xlim([water_year[0], water_year[-1]])
ax.set_xlabel('Water Year 2019')
ax.set_ylabel('Albedo')
ax.set_title(None);